In [ ]:
%load_ext autoreload
%autoreload 2

from skorecard.reporting import create_report 
from skorecard import datasets

import pandas as pd
import numpy as np

In [ ]:
X, y = datasets.load_uci_credit_card(return_X_y=True)

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score

from skorecard.bucketers import DecisionTreeBucketer, EqualWidthBucketer, OrdinalCategoricalBucketer

from sklearn.linear_model import LogisticRegression

from dabl import detect_types
# using make pipeline

detected_types = detect_types(X)
cat_columns = X.columns[(detected_types['categorical']==True) | (detected_types['low_card_int']==True)]
num_columns = X.columns[(detected_types['continuous']==True) | (detected_types['dirty_float']==True)]


bucket_pipeline = make_pipeline(
    EqualWidthBucketer(bins=5, variables=list(num_columns)),
    OrdinalCategoricalBucketer(variables=list(cat_columns))
)

pipeline = Pipeline([
    ('bucketing', bucket_pipeline),
    ('one-hot-encoding', OneHotEncoder()),
    ('lr', LogisticRegression())
])

pipeline.fit(X, y)
f"AUC = {roc_auc_score(y, pipeline.predict_proba(X)[:,1]):.4f}"


In [ ]:
df = create_report(X, y, column='LIMIT_BAL', bucketer=bucket_pipeline.named_steps['equalwidthbucketer'])